In [1]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.datasets import load_breast_cancer
from random import seed
from random import randrange
from csv import reader
from math import sqrt
from sklearn import preprocessing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prettytable import PrettyTable
from sklearn.linear_model import SGDClassifier
from sklearn import preprocessing
from sklearn.metrics import log_loss
import math

In [2]:
X = load_breast_cancer().data
Y = load_breast_cancer().target

In [3]:
X_df = pd.DataFrame(X)
Y_df = pd.DataFrame(Y)

In [4]:
X_df.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [5]:
Y_df.head()

,0
0,0
1,0
2,0
3,0
4,0


In [6]:
print(X_df.shape)
print(Y_df.shape)

(569, 30)
(569, 1)


In [7]:
#splitting the data into train set and test set

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_df, Y_df, test_size = 0.3, random_state = 5)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(398, 30)
(171, 30)
(398, 1)
(171, 1)


In [8]:
scalar = preprocessing.StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test=scalar.transform(X_test)

X_train_df = pd.DataFrame(X_train)
Y_train_df = pd.DataFrame(Y_train) 

In [9]:
Y_train_df.reset_index(drop=True,inplace=True)
Y_train_df.head()

,0
0,0
1,1
2,0
3,1
4,1


In [10]:
X_train_df['class'] = Y_train_df
X_train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,class
0,-0.280700,-0.107141,-0.251667,-0.353201,1.376494,0.402148,0.469188,0.655619,1.173114,0.818014,...,0.414375,-0.129156,-0.211767,2.034236,1.093524,1.155073,1.454746,1.673402,1.927756,0
1,-0.524643,-1.625403,-0.551996,-0.552417,-0.464193,-0.758859,-0.726158,-0.573343,-0.418000,-0.657791,...,-1.033312,-0.613384,-0.578254,-0.143060,-0.267148,-0.430995,-0.103326,0.363454,-0.418850,1
2,-0.042636,0.120955,-0.032393,-0.157602,0.676504,0.212612,0.340684,0.425958,-0.490488,0.463150,...,0.889710,0.215834,-0.004688,1.271742,0.704575,0.692819,1.055279,0.477867,1.234618,0
3,-1.358163,-1.585012,-1.345477,-1.138915,0.419365,-0.723080,-0.726415,-0.722872,0.038676,1.012960,...,-1.635403,-1.299944,-1.049317,-0.002021,-0.869434,-0.877117,-0.787928,-0.730933,-0.317674,1
4,-0.365933,2.330633,-0.400125,-0.414383,-1.092754,-0.852274,-0.310190,-0.652952,-0.863801,-0.810092,...,1.713624,-0.563034,-0.520219,-0.429547,-0.562437,-0.111315,-0.747216,-0.860269,-0.614494,1


In [11]:
W = np.random.rand(1,30) #choosing a random value of W
iteration = 800   #setting the iteration count to 800 for SGD
learn_rate = 1 #initializing learning rate to 1
lamda = 1    #picking lamda value of 1 in L2 regularization
k = 30      #picking batchsize of 30  

for p in range(iteration):
    w,temp_vector=W,0
    data=X_train_df.sample(30) #sampling random k=batch size=25 data ; https://www.geeksforgeeks.org/python-random-sample-function/
    x=np.array(data.drop('class',axis=1))
    y=np.array(data['class'])
    
    for i in range(k):
        temp_vector += (-y[i] * x[i] * math.exp(-np.dot(y[i],np.dot(w,x[i]))))/(1+math.exp(-np.dot(y[i],np.dot(w,x[i])))) + 2 * lamda * W
        W=(w-learn_rate*(temp_vector))
        
    learn_rate = learn_rate/2 
    iteration -= 1    
    
    
print(W)  

[[-0.05577926 -0.03795673 -0.05693249 -0.05716765 -0.03619538 -0.04566405
  -0.05137774 -0.05900076 -0.03396192  0.00039323 -0.05725254 -0.01527104
  -0.05968474 -0.05878624 -0.01307262 -0.02775585 -0.01002757 -0.030606
  -0.01958256 -0.00227324 -0.05977959 -0.03931996 -0.06043787 -0.06005506
  -0.03077441 -0.04121526 -0.04537893 -0.05073427 -0.0330751  -0.02216779]]


In [12]:
y_predictions = []
for i in range(len(X_test)):
    try:
        prediction = 1/(1+math.exp(-np.dot(W,X_test[i])))
    except OverflowError:
        prediction = float('inf')
        
    y_predictions.append(prediction)    

In [13]:
from sklearn.metrics import log_loss
log_loss(Y_test, y_predictions)

0.43444028437283283

In [14]:
from sklearn.linear_model import SGDClassifier
model = SGDClassifier(loss = 'log',penalty ='l2')
model.fit(X_train,Y_train)
y_sgd_predictions = model.predict(X_test)

In [15]:
log_loss(Y_test, y_sgd_predictions)

0.4039716490332276

In [16]:
from prettytable import PrettyTable    
x = PrettyTable()
x.field_names=['Weight Vectors obtained from Manual Implementation','Weight vector obtained from SGD sklearn']
weight_sgd=model.coef_ #https://datascience.stackexchange.com/questions/17970/how-can-i-know-how-to-interpret-the-output-coefficients-coefs-from-the-mode
for i in range(29):
    x.add_row([W[0][i],weight_sgd[0][i]])
print(x)

+----------------------------------------------------+-----------------------------------------+
| Weight Vectors obtained from Manual Implementation | Weight vector obtained from SGD sklearn |
+----------------------------------------------------+-----------------------------------------+
|               -0.055779255088234274                |           -14.766490045582394           |
|                -0.03795673117202592                |           -11.182810974668651           |
|               -0.056932489637636294                |           -14.264631476896083           |
|               -0.057167649308175084                |           -15.209228377345203           |
|                -0.03619538453316427                |            -3.91733555382231            |
|               -0.045664052669209663                |            10.124250658244422           |
|                -0.05137774192889916                |           -16.321005960893544           |
|               -0.05900075768

In [17]:
print(type(y_sgd_predictions))

<class 'numpy.ndarray'>


In [18]:
print(type(y_predictions))

<class 'list'>


In [19]:
y_predictions = [ 1 if x > 0.5 else 0 for x in y_predictions]    

In [20]:
from sklearn.metrics import accuracy_score
print(accuracy_score(Y_test,y_predictions))

0.9239766081871345


In [21]:
print(accuracy_score(Y_test,y_sgd_predictions))

0.9883040935672515


Observation: So the manually implemented SGD and scikit learn SGD classifier produced similar accuracy score results of around 92 and 98 percent respectively.